<a href="https://www.kaggle.com/phlynn/more-features-plz?scriptVersionId=90037985" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# We need more features, right?

# I was going to use this additional data to train model with
# but did not have time....
# This notebook shows you how to get college baksetball player data
# from sports reference. Player data can yield multi-season features
# based on player experience as well as coaching experience. 
# Moreover, the player dynamic between two teams could be analyzed 
# a-priori to the game being played.

In [2]:
!git clone https://github.com/tflynn3/mbball-player-scraper.git

Cloning into 'mbball-player-scraper'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 43 (delta 21), reused 32 (delta 13), pack-reused 0
Unpacking objects: 100% (43/43), 115.35 KiB | 690.00 KiB/s, done.


In [3]:
import os
os.chdir('mbball-player-scraper')

In [4]:
from scraper import get_all_schools, get_roster, get_player_gamelog
import pandas as pd
from pprint import pprint

In [5]:
# Create a schools dataframe
s = get_all_schools()
schools_df = pd.DataFrame(s)
print(schools_df.head())

                  school_name                      school_link  \
0                         NaN                              NaN   
1  Abilene Christian Wildcats  /cbb/schools/abilene-christian/   
2           Air Force Falcons          /cbb/schools/air-force/   
3                  Akron Zips              /cbb/schools/akron/   
4        Alabama A&M Bulldogs         /cbb/schools/alabama-am/   

                 location year_min year_max years     g wins losses  \
0                     NaN      NaN      NaN   NaN   NaN  NaN    NaN   
1          Abilene, Texas     1971     2022    12   356  195    161   
2  USAF Academy, Colorado     1958     2022    65  1780  771   1009   
3             Akron, Ohio     1902     2022    71  1649  981    668   
4         Normal, Alabama     2000     2022    23   655  250    405   

  win_loss_pct     srs     sos poll_final conf_champ_count  \
0          NaN     NaN     NaN        NaN              NaN   
1         .548   -8.41   -6.47          0           

In [6]:
# Get Jayhawks
my_school = schools_df.loc[schools_df['school_name'] == 'Kansas Jayhawks'].to_dict('records')[0]
pprint(my_school)

# Check if school played this year
if my_school['year_max'] == '2022':
    print(f'\nThe {my_school["school_name"]} played this year!')


{'conf_champ_count': '63',
 'conf_champ_post_count': '15',
 'g': '3228',
 'location': 'Lawrence, Kansas',
 'losses': '877',
 'ncaa_champ_count': '3',
 'ncaa_count': '49',
 'ncaa_final_four_count': '15',
 'poll_final': '46',
 'school_link': '/cbb/schools/kansas/',
 'school_name': 'Kansas Jayhawks',
 'sos': '7.25',
 'srs': '15.83',
 'win_loss_pct': '.728',
 'wins': '2351',
 'year_max': '2022',
 'year_min': '1899',
 'years': '124'}

The Kansas Jayhawks played this year!


In [7]:
# Get team roster
ku_roster = get_roster(my_school["school_link"])
roster_df = pd.DataFrame(ku_roster)
print(roster_df.head())

Season: 2022
/cbb/schools/kansas/2022.html
Could not get player link...skipping player
            player                          player_link number class pos  \
0     Ochai Agbaji     /cbb/players/ochai-agbaji-1.html     30    SR   G   
1  Christian Braun  /cbb/players/christian-braun-1.html      2    JR   G   
2  David McCormack  /cbb/players/david-mccormack-1.html     33    SR   F   
3     Jalen Wilson     /cbb/players/jalen-wilson-1.html     10    SO   F   
4    Dajuan Harris    /cbb/players/dajuan-harris-1.html      3    JR   G   

  height weight           hometown               high_school       rsci  \
0    6-5    210    Kansas City, MO               Oak Park HS              
1    6-6    205  Overland Park, KS  Blue Valley Northwest HS              
2   6-10    265        Norfolk, VA          Oak Hill Academy  30 (2018)   
3    6-8    215         Denton, TX          John H. Guyer HS  51 (2019)   
4    6-1    160       Columbia, MO            Rock Bridge HS              

     

In [8]:
# Initialize a player games array
ku_player_game_stats = []

# A gamelog is a list of all games a certain player has played
# regardless of what season it was
# Now let's get the gamelogs from KU's current roster
for ku_player in ku_roster:
    # Get player game log
    p = get_player_gamelog(ku_player['player_link'])
    # Player gamelog has some empty rows
    ku_player_game_stats += p

ku_gamelog_df = pd.DataFrame(ku_player_game_stats).dropna()
ku_gamelog_df


,season,date_game,school_name,game_location,opp_name,game_type,game_result,gs,mp,fg,...,ft_pct,orb,drb,trb,ast,stl,blk,tov,pf,pts
0,2018-19,2019-01-09,Kansas,,TCU,REG,W,0,25,3,...,,1,3,4,1,2,1,0,2,7
1,2018-19,2019-01-12,Kansas,@,Baylor,REG,W,0,21,4,...,.500,1,2,3,0,1,0,0,2,10
2,2018-19,2019-01-14,Kansas,,Texas,REG,W,0,25,2,...,,0,4,4,2,1,0,0,1,4
3,2018-19,2019-01-19,Kansas,@,West Virginia,REG,L,0,4,0,...,,1,2,3,0,0,0,3,0,0
4,2018-19,2019-01-21,Kansas,,Iowa State,REG,W,0,16,3,...,,1,4,5,0,0,1,2,5,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1173,2017-18,2018-02-15,Jacksonville State,@,Murray State,REG,L,0,2,0,...,,0,0,0,0,0,0,0,1,0
1174,2017-18,2018-02-17,Jacksonville State,@,Austin Peay,REG,L,0,26,5,...,.700,4,6,10,2,1,2,0,3,18
1175,2017-18,2018-02-22,Jacksonville State,,Belmont,REG,W,0,20,3,...,1.000,3,7,10,4,0,0,0,3,10
1176,2017-18,2018-03-01,Jacksonville State,N,Tennessee Tech,CTOURN,W,0,8,0,...,,0,1,1,1,0,0,1,2,0


In [9]:
# Let's review data...
ku_gamelog_df.to_csv('go_hawks.csv')

In [10]:
# Looks like the correct data, but it needs to joined with previous data frames
# I'm gonna repeat code...

# Initialize a player games array
ku_player_game_stats = []

# A gamelog is a list of all games a certain player has played
# regardless of what season it was
# Now let's get the gamelogs from KU's current roster
for ku_player in ku_roster:
    # Get player game log
    p = get_player_gamelog(ku_player['player_link'])
    # Player gamelog has some empty rows
    for game in p:
        ku_player_game = {}
        # add player data
        ku_player_game.update(ku_player)
        # add game data
        ku_player_game.update(game)
        ku_player_game_stats.append(ku_player_game)

ku_gamelog_df = pd.DataFrame(ku_player_game_stats).dropna()
ku_gamelog_df

,player,player_link,number,class,pos,height,weight,hometown,high_school,rsci,...,ft_pct,orb,drb,trb,ast,stl,blk,tov,pf,pts
0,Ochai Agbaji,/cbb/players/ochai-agbaji-1.html,30,SR,G,6-5,210,"Kansas City, MO",Oak Park HS,,...,,1,3,4,1,2,1,0,2,7
1,Ochai Agbaji,/cbb/players/ochai-agbaji-1.html,30,SR,G,6-5,210,"Kansas City, MO",Oak Park HS,,...,.500,1,2,3,0,1,0,0,2,10
2,Ochai Agbaji,/cbb/players/ochai-agbaji-1.html,30,SR,G,6-5,210,"Kansas City, MO",Oak Park HS,,...,,0,4,4,2,1,0,0,1,4
3,Ochai Agbaji,/cbb/players/ochai-agbaji-1.html,30,SR,G,6-5,210,"Kansas City, MO",Oak Park HS,,...,,1,2,3,0,0,0,3,0,0
4,Ochai Agbaji,/cbb/players/ochai-agbaji-1.html,30,SR,G,6-5,210,"Kansas City, MO",Oak Park HS,,...,,1,4,5,0,0,1,2,5,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1173,Cam Martin,/cbb/players/cam-martin-1.html,31,SR,F,6-9,230,"Yukon, OK",Yukon High School,,...,,0,0,0,0,0,0,0,1,0
1174,Cam Martin,/cbb/players/cam-martin-1.html,31,SR,F,6-9,230,"Yukon, OK",Yukon High School,,...,.700,4,6,10,2,1,2,0,3,18
1175,Cam Martin,/cbb/players/cam-martin-1.html,31,SR,F,6-9,230,"Yukon, OK",Yukon High School,,...,1.000,3,7,10,4,0,0,0,3,10
1176,Cam Martin,/cbb/players/cam-martin-1.html,31,SR,F,6-9,230,"Yukon, OK",Yukon High School,,...,,0,1,1,1,0,0,1,2,0


In [11]:
# Let's review data... again....
ku_gamelog_df.to_csv('go_hawks_again.csv')